Data Cleaning

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# read the csv-file
df = pd.read_csv('../data/raw/credit-risk-dataset/credit_risk_dataset.csv')

# check
display(df.head())
df.dtypes

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


person_age                      int64
person_income                   int64
person_home_ownership          object
person_emp_length             float64
loan_intent                    object
loan_grade                     object
loan_amnt                       int64
loan_int_rate                 float64
loan_status                     int64
loan_percent_income           float64
cb_person_default_on_file      object
cb_person_cred_hist_length      int64
dtype: object

Datatypes

In [3]:
# convert 'object' datatype into 'category'
cat_cols = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']

for col in cat_cols:
    df[col] = df[col].astype('category')

# check
df[cat_cols].dtypes

person_home_ownership        category
loan_intent                  category
loan_grade                   category
cb_person_default_on_file    category
dtype: object

Missing Values

In [4]:
# replace missing values of column 'person_emp_length' with median

median_emp_length = df['person_emp_length'].median()
df.fillna({'person_emp_length': median_emp_length}, inplace=True)

In [5]:
df.groupby('loan_grade', observed=False)['loan_int_rate'].median()

loan_grade
A     7.490
B    10.990
C    13.480
D    15.310
E    16.820
F    18.535
G    20.160
Name: loan_int_rate, dtype: float64

In [6]:
# replace missing values of column 'loan_int_rate' with median per 'loan_grade'
df['loan_int_rate'] = df.groupby('loan_grade', observed=False)['loan_int_rate'].transform(lambda x: x.fillna(x.median()))

# check missing values
df.isna().sum()

person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_status                   0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
dtype: int64

Unrelistic values

In [7]:
# clean up unrealistic values
df = df[df['person_age'] <= 100]
df = df[df['person_emp_length'] <= 50]
df = df[df['person_income'] <= 1000000]

df.shape

(32566, 12)

In [8]:
# change the column name
df.rename(columns={'cb_person_default_on_file': 'default'}, inplace=True)
df.rename(columns={'cb_person_cred_hist_length': 'credit_history'}, inplace=True)

print(df.shape)
df.head()

(32566, 12)


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,default,credit_history
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4
5,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,0.25,N,2


In [9]:
# check statistics, unrealisic values
df.describe()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,credit_history
count,32566.000000,32566.000000,32566.000000,32566.000000,32566.000000,32566.000000,32566.000000,32566.000000
mean,27.714058,65522.380704,4.760149,9588.316803,11.014286,0.218234,0.170242,5.801449
std,6.197416,47103.173555,3.980636,6320.788064,3.212277,0.413053,0.106737,4.049647
min,20.000000,4000.000000,0.000000,500.000000,5.420000,0.000000,0.000000,2.000000
25%,23.000000,38500.000000,2.000000,5000.000000,7.880000,0.000000,0.090000,3.000000
50%,26.000000,55000.000000,4.000000,8000.000000,10.990000,0.000000,0.150000,4.000000
75%,30.000000,79200.000000,7.000000,12200.000000,13.480000,0.000000,0.230000,8.000000
max,94.000000,948000.000000,41.000000,35000.000000,23.220000,1.000000,0.830000,30.000000


Inconsistencies

In [10]:
# check calculation of of the ratio
ratio = (df['loan_amnt'] / df['person_income']).round(2)
mask_ratio = ratio != df['loan_percent_income']

print(df.loc[mask_ratio,:].shape)
df.loc[mask_ratio,:]

(1010, 12)


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,default,credit_history
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4
17,23,92111,RENT,7.0,MEDICAL,F,35000,20.25,1,0.32,N,4
36,22,48000,RENT,1.0,EDUCATION,E,30000,18.39,1,0.63,N,2
40,26,62050,RENT,6.0,MEDICAL,E,30000,17.99,1,0.41,N,2
48,22,66300,RENT,4.0,MEDICAL,B,30000,12.69,1,0.38,N,3
...,...,...,...,...,...,...,...,...,...,...,...,...
32480,62,62050,MORTGAGE,29.0,MEDICAL,C,25000,13.43,1,0.34,Y,28
32485,64,24000,RENT,1.0,MEDICAL,A,3000,6.99,0,0.13,N,21
32542,67,200000,MORTGAGE,2.0,PERSONAL,C,3000,10.28,0,0.01,N,26
32555,51,80000,RENT,3.0,HOMEIMPROVEMENT,F,18000,18.78,0,0.23,Y,20


1010 values in column 'loan_int_rate' are calculated wrong

In [ ]:
# rewrite column with the correct values
df.loc[mask_ratio, 'loan_percent_income'] = ratio[mask_ratio]

# check
mask_ratio = ratio != df['loan_percent_income']
print(df.loc[mask_ratio,:].shape)

(0, 12)


Save cleaned dataset

In [13]:
df.to_csv('../data/processed/credit_risk_cleaned.csv', index=False)